# Assignment 3

In [51]:
import pandas as pd
import requests
import urllib.request, urllib.parse, urllib.error
import re
from datetime import timedelta

api_key = 'AIzaSyCYNp4jA________vdO6VSklFq_UnJI'

In [52]:
# read the text file into a dataframe
df = pd.read_csv("C:/Users/ariel/Desktop/מטלה 3/dests.txt", header = None)
df.columns = ['Target']

In [53]:
def Distance_Duration(destination):
    serviceurl="https://maps.googleapis.com/maps/api/distancematrix/json?"
    parms = dict() 
    parms['origins'] = 'Tel Aviv'
    parms['destinations'] = destination
    parms['key'] = api_key
    url = serviceurl + urllib.parse.urlencode(parms)
    
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
                return (response_data['rows'][0]['elements'][0]['distance']['text'],
                        response_data['rows'][0]['elements'][0]['duration']['text'])
                      
            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
  



In [54]:
def get_lat_lng(destination):
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (destination,api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            print("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
                
                return (response_data['results'][0]["geometry"]["location"]["lng"],
                       response_data['results'][0]["geometry"]["location"]["lat"])

            except:
                print("Response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")

In [55]:
def get_info_row(row):
    # call your function to get the distance, duration, longitude, and latitude for the target in the row
    distance, duration = Distance_Duration(row['Target'])
    longitude, latitude = get_lat_lng(row['Target'])
    
    # return a Series with the distance, duration, longitude, and latitude
    return pd.Series({'Distance_km': distance, 'Duration (hour + minutes)': duration, 'Longitude': longitude, 'Latitude': latitude})

# apply the get_distance_info_row function to each row of the Target column, and concatenate the results with the original dataframe
df = pd.concat([df, df.apply(get_info_row, axis=1)], axis=1)


In [56]:
def convert_duration_to_hours_minutes(duration):
    duration_parts = duration.split()
    days = hours = minutes = 0
    for i in range(len(duration_parts)):
        if duration_parts[i] == 'days' or duration_parts[i] == 'day':
            days = int(duration_parts[i-1])
        elif duration_parts[i] == 'hours' or duration_parts[i] == 'hour':
            hours = int(duration_parts[i-1])
        elif duration_parts[i] == 'mins' or duration_parts[i] == 'min':
            minutes = int(duration_parts[i-1])
    total_seconds = timedelta(days=days, hours=hours, minutes=minutes).total_seconds()
    hours = int(total_seconds // 3600)
    minutes = int(total_seconds % 3600 // 60)
    return f"{hours} hours {minutes} minutes"


In [57]:
df['Duration (hour + minutes)'] = df['Duration (hour + minutes)'].apply(convert_duration_to_hours_minutes)
df

,Target,Distance_km,Duration (hour + minutes),Longitude,Latitude
0,Istanbul,"1,815 km",21 hours 3 minutes,28.978359,41.008238
1,Amsterdam,"4,533 km",48 hours 0 minutes,4.904139,52.367573
2,Valletta,"3,793 km",51 hours 0 minutes,14.514100,35.899237
3,Basel,"4,093 km",44 hours 0 minutes,7.588576,47.559599
4,Doha,"2,164 km",22 hours 39 minutes,51.531040,25.285447


## 3 Cities Furthest From Tel Aviv - From Top to Bottom 

In [58]:
def convert (Distance):
    pattern = r"[^0-9.]"
    return re.sub(pattern, "", Distance)
df['Distance_km'] = df['Distance_km'].apply(convert) 
# convert 'Distance_km' column to a numeric data type
df['Distance_km'] = pd.to_numeric(df['Distance_km'])

top_three = df.nlargest(3, 'Distance_km').reset_index(drop=True)
top_three.index = top_three.index + 1
top_three

,Target,Distance_km,Duration (hour + minutes),Longitude,Latitude
1,Amsterdam,4533,48 hours 0 minutes,4.904139,52.367573
2,Basel,4093,44 hours 0 minutes,7.588576,47.559599
3,Valletta,3793,51 hours 0 minutes,14.514100,35.899237
